In [46]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
datas = load_iris()
dir(datas)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [4]:
df = pd.DataFrame(datas.data, columns=datas.feature_names)
df['target'] = datas.target
df['target_name'] = df['target'].apply(lambda x: datas.target_names[x])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


Suppose we have to predict the target variable. to do tha twe have many models. But how we can find a best fitting model that gives us the best accurecy? 
We havae already answred this question with K fold cross validation. But it's not enough. Because we have to do manually the parametre tuning. suppose you want to try 5 different model. those 5 model have many parametres that can have various types of values. So it becomes so difficult for us to try every single parametre by ourselves. thats why we will use grid serach cv method to do all these task by the computer itself.

At first let's do it in old way

In [18]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['target', 'target_name'], axis=1), df.target, test_size=0.2)

In [19]:
model = SVC(kernel='rbf', C=30, gamma='auto')
model.fit(x_train, y_train)

SVC(C=30, gamma='auto')

In [20]:
model.score(x_test, y_test)

0.9666666666666667

Now let's apply cross validation for this model with different parametre values and find the best parametre value combination

In [25]:
cross_val_score(SVC(kernel='linear', C=10, gamma='auto'), datas.data, datas.target, cv=5).mean()

0.9733333333333334

In [26]:
cross_val_score(SVC(kernel='rbf', C=10, gamma='auto'), datas.data, datas.target, cv=5).mean()

0.9800000000000001

In [27]:
cross_val_score(SVC(kernel='linear', C=20, gamma='auto'), datas.data, datas.target, cv=5).mean()

0.9666666666666666

Now these parametre couldn't have only thes values and SVC doesn't have only these 3 parapetre. and we are not just trying SVC for this data set. So can u imagine how much work we have to do?

for this problem we can use loops. But the prb here is the higher the parametre number, the highe the numbers of loop not the number of iteration. you can see this in 15_grid_search.ipynb

Grid Seacrh CV does the same thing for us. without any hassle. let's try it

In [31]:
clf = GridSearchCV(SVC(gamma='auto'),
                   {
                       'kernel' : ['rbf', 'linear'],
                       'C' : [1,10,20]
                   }, cv=5, return_train_score=False)

clf.fit(datas.data, datas.target)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']})

In [32]:
df_result = pd.DataFrame(clf.cv_results_)
df_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002398,0.000489,0.001599,0.000490,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001399,0.000490,0.001199,0.000748,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000600,0.000490,0.000600,0.000490,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000801,0.000401,0.000799,0.000399,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000999,0.000001,0.000409,0.000501,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000800,0.000748,0.000600,0.000490,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [33]:
df_result = df_result[['params','mean_test_score']]
df_result

,params,mean_test_score
0,"{'C': 1, 'kernel': 'rbf'}",0.980000
1,"{'C': 1, 'kernel': 'linear'}",0.980000
2,"{'C': 10, 'kernel': 'rbf'}",0.980000
3,"{'C': 10, 'kernel': 'linear'}",0.973333
4,"{'C': 20, 'kernel': 'rbf'}",0.966667
5,"{'C': 20, 'kernel': 'linear'}",0.966667


In [34]:
clf.best_score_

0.9800000000000001

In [35]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

This is not the end. This gridsearchcv can also have issues with computational time and error when you have low computational power and millions of data. We face this problem in our practical life projects. So to avoid this we use randomized search cv. This doesn't try every singele parametre combination. it takes random parametre combination

In [44]:
rs = RandomizedSearchCV(SVC(gamma='auto'),
                   {
                       'kernel' : ['rbf', 'linear'],
                       'C' : [1,10,20]
                   }, cv=5, return_train_score=False, n_iter=2)

rs.fit(datas.data, datas.target)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [1, 10, 20],
                                        'kernel': ['rbf', 'linear']})

In [45]:
df_result2 = pd.DataFrame(rs.cv_results_)[['params','mean_test_score']]
df_result2

,params,mean_test_score
0,"{'kernel': 'rbf', 'C': 20}",0.966667
1,"{'kernel': 'rbf', 'C': 1}",0.980000


Now let's try grid search cv for various models

at first we have to define the parametres we want to use for each type of model

In [47]:
model_params = {
    'svm' : {
        'model': SVC(gamma='auto'),
        'params': {
            'kernel': ['rbf','linear'],
            'C': [1,10,20]
        }
    },
    'LogisticRegression' : {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params':{
            'C': [1,5,10]
        }
    },
    'random_forrest':{
        'model': RandomForestClassifier(),
        'params':{
            'n_estimators': [10,40,50]
        }
    }
}

Now we will run a for loop

In [48]:
for model_name, mdl in model_params.items():
    print(model_name, mdl)

svm {'model': SVC(gamma='auto'), 'params': {'kernel': ['rbf', 'linear'], 'C': [1, 10, 20]}}
LogisticRegression {'model': LogisticRegression(multi_class='auto', solver='liblinear'), 'params': {'C': [1, 5, 10]}}
random_forrest {'model': RandomForestClassifier(), 'params': {'n_estimators': [10, 40, 50]}}


In [50]:
scores = []
for i, j in model_params.items():
    clf = GridSearchCV(j['model'], j['params'], cv=5, return_train_score=False)
    clf.fit(datas.data, datas.target)
    scores.append({
        'model': i,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\

In [51]:
scores

[{'model': 'svm',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 1, 'kernel': 'rbf'}},
 {'model': 'LogisticRegression',
  'best_score': 0.9666666666666668,
  'best_params': {'C': 5}},
 {'model': 'random_forrest',
  'best_score': 0.96,
  'best_params': {'n_estimators': 50}}]

In [52]:
final_df = pd.DataFrame(scores, columns=['model','best_score','best_params'])
final_df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,LogisticRegression,0.966667,{'C': 5}
2,random_forrest,0.960000,{'n_estimators': 50}
